In [ ]:
import networkx as nx

# Define the k-clique size for this experiment
k = 100  # Adjust this value as needed
# Define "big"
bigFactor = 5

# Define clique sizes
sizeBig = bigFactor*k

# Create an empty graph
G = nx.Graph()

# Create and add nodes for each clique
clique_A = [f"A{i}" for i in range(sizeBig)]
cliqueChain = [f"B{i}" for i in range(k)]
clique_C = [f"C{i}" for i in range(sizeBig)]

for clique in [clique_A, cliqueChain, clique_C]:
  G.add_nodes_from(clique)
  # Add edges within each clique to create complete subgraphs
  G.add_edges_from([(n1, n2) for n1 in clique for n2 in clique if n1 != n2])

# Make a big chain of k-cliques
for i in range(sizeBig-k):
  # New node
  n = f"N{i}"
  G.add_node(n)
  # Connect to k-1 previous nodes
  G.add_edges_from([(n, n2) for n2 in cliqueChain[-(k-1):]])
  # Add the new node to the list
  cliqueChain.append(n)

# Make pairs of nodes from clique A and the beginning of the chain
nodes_A = clique_A[:k-2]
nodes_Ba = cliqueChain[:k-2]
AB_pairs = list(zip(nodes_A, nodes_Ba))

# Same for B and C
nodes_Bc = cliqueChain[-(k-2):]
nodes_C = clique_C[:k-2]
BC_pairs = list(zip(nodes_Bc, nodes_C))

# Add them as pairs to merge
pairs_to_merge = AB_pairs + BC_pairs

# Function to merge two nodes
def merge_nodes(graph, node1, node2):
    # Add edges incident to node1 to node2
    edges_to_add = [(node2, neighbor) for neighbor in graph.neighbors(node1)]
    graph.add_edges_from(edges_to_add)

    # Remove node1
    graph.remove_node(node1)

# Merge nodes from the pairs_to_merge list
for pair in pairs_to_merge:
    merge_nodes(G, pair[0], pair[1])

In [ ]:
from collections import defaultdict

# Compute k-clique communities
communities = list(nx.algorithms.community.k_clique_communities(G, k))

# Map the nodes to their communities
mapping = defaultdict(lambda: 'None')

for i, community in enumerate(communities):
    value = 'Community ' + str(i)
    for node in community:
        if mapping[node] == 'None':
            mapping[node] = value
        else:
            mapping[node] = 'Multiple'  # If a node is in multiple communities

nx.set_node_attributes(G, mapping, 'community')

In [ ]:
# Export the graph as a GraphML file
nx.write_graphml(G, "cluster_as_bridge.graphml")

In [ ]:
# Visualize the network
import matplotlib.pyplot as plt

# Apply Force-directed layout
pos = nx.spring_layout(G, scale=10.0, iterations=250)

# Draw the graph with nodes colored according to the "community" attribute
community_colors = {'None':'lightgrey', 'Community 0': 'red', 'Community 1': 'blue', 'Community 2': 'green', 'Multiple': 'black'}
node_colors = [community_colors[mapping[node]] for node in G.nodes()]

# Draw the graph with equal aspect ratio
plt.figure(figsize=(8, 8))
nx.draw(G, pos, with_labels=False, node_size=10, node_color=node_colors, edge_color="lightgrey")
plt.gca().set_aspect('equal', adjustable='box')
plt.show()